# Length

In [34]:
import os
import pandas
import numpy as np
import pandas as pd
import re
import ast
import json
from CS_feature_extractor import *
DATA_DIR = '../data'

# Experiment Config
DF_NAME = 'BigBench'

NUM_OF_SAMPLES = 500
NUM_OF_COT = 40
MODEL = 'gpt-3.5-turbo-0125'
storage_dir = os.path.join(DATA_DIR, f'Evaluation_CoTs/Algo_Design_Data')
hard_file_path = os.path.join(storage_dir, f'{DF_NAME}_hard.json')
easy_file_path = os.path.join(storage_dir, f'{DF_NAME}_easy.json')

In [35]:
def sigmoid(x):
    return 1 / (1 + np.exp(-(x-5)))

In [36]:
with open(hard_file_path,'r') as f:
    data = json.load(f)
hard = pd.DataFrame(data)
hard['size_of_cot'] = [[(i+1)^2 for i in range(40)] for j in range(500)]
for row_idx in range(len(hard)):
    hard['SIM_COT_AGG'][row_idx][0] = 0.5
# hard['SIM_COT_AGG'] = hard['SIM_COT_AGG'].apply(lambda lst: [x*sigmoid(idx) for idx,x in enumerate(lst)])
hard

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,A,"[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.9513888888888888, 0.8311688311688312, ...","[0.5, 0.9513888888888888, 0.8205128205128205, ...","[0.0, 0.9513888888888888, 0.8273491214667685, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
1,1,C,"[B, B, C, C, C, A, C, C, C, C, C, C, B, C, C, ...","[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1, 4, 3, 3, 4, 5, 3, 4, 3, 1, 3, 3, 4, 3, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 0, 0, 9, 0, 9, 2, 0, 2, 2, 0, 0, 2, 0, ...",disambiguation_qa,"[0.0, 0.5139664804469274, 0.6190476190476191, ...","[0.5, 0.5139664804469274, 0.5431472081218274, ...","[0.0, 0.5139664804469274, 0.5462102000563539, ...","[0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
2,2,C,"[ERROR, B, B, B, B, C, B, ERROR, B, B, B, B, B...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 4, 4, 4, 3, 4, 4, 0, 3, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 0, 0, 2, 0, 2, 9, 10, 0, 0, 0, 5, 0, 9, 0...",disambiguation_qa,"[0.0, 0.018292682926829285, 0.8596491228070176...","[0.5, 0.024390243902439046, 0.8546511627906976...","[0.0, 0.018292682926829285, 0.4394399460188934...","[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
3,3,C,"[C, B, C, C, B, C, C, C, C, B, B, B, C, C, C, ...","[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[1, 4, 1, 1, 4, 1, 1, 1, 1, 5, 4, 4, 1, 3, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.4204545454545454, 0.4204545454545454, ...","[0.5, 0.4204545454545454, 0.6079545454545454, ...","[0.0, 0.4204545454545454, 0.7102272727272727, ...","[0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, ...","[0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
4,4,C,"[C, C, C, C, C, C, C, C, C, C, C, C, B, A, C, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[4, 4, 5, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 5, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.8661971830985915, 0.6488095238095238, ...","[0.5, 0.8661971830985915, 0.6098901098901099, ...","[0.0, 0.8661971830985915, 0.6299603174603174, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,A,"[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 5, 4, 5, 5, 4, 5, 4, 5, 5, 4, 4, 3, 4, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",movie_recommendation,"[0.0, 0.72, 0.8153846153846154, 0.739583333333...","[0.5, 0.72, 0.7718446601941747, 0.757009345794...","[0.0, 0.72, 0.8009507346585998, 0.792078305681...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [37]:
with open(easy_file_path,'r') as f:
    data = json.load(f)
easy = pd.DataFrame(data)
easy['size_of_cot'] = [[i+1 for i in range(40)] for j in range(500)]
for row_idx in range(len(easy)):
    easy['SIM_COT_AGG'][row_idx][0] = 0.5
easy['SIM_COT_AGG'] = easy['SIM_COT_AGG'].apply(lambda lst: [x*sigmoid(idx) for idx,x in enumerate(lst)])
easy

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,A,"[B, A, B, B, B, A, B, B, A, B, B, B, B, B, B, ...","[0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.8023952095808383, 0.8098159509202454, ...","[0.0033464254621424277, 0.014432048712097419, ...","[0.0, 0.8023952095808383, 0.8303738139694394, ...","[0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,1,C,"[A, A, A, A, B, A, A, A, A, A, B, A, A, A, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 2, 5, 4, 3, 3, 4, 1, 4, 1, 3, 3, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.6206896551724138, 0.6206896551724138, ...","[0.0033464254621424277, 0.011287897286554011, ...","[0.0, 0.6206896551724138, 0.8103448275862069, ...","[0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, ...","[0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,2,C,"[C, C, A, C, A, A, C, C, A, C, C, C, C, C, A, ...","[1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, ...","[4, 4, 4, 4, 1, 1, 4, 2, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.8165680473372781, 0.7604790419161677, ...","[0.0033464254621424277, 0.014686964347743402, ...","[0.0, 0.8165680473372781, 0.7885235446267229, ...","[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, ...","[0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, ...","[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,3,C,"[A, B, B, B, A, A, A, A, B, A, B, B, A, B, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 3, 1, 5, 1, 2, 1, 1, 2, 2, 4, 2, 3, 4, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.5859872611464968, 0.6025641025641026, ...","[0.0033464254621424277, 0.010539689914091868, ...","[0.0, 0.5859872611464968, 0.6611305361305362, ...","[0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, ...","[0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,4,C,"[C, B, C, C, C, A, B, C, C, C, A, C, C, C, C, ...","[1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.5302013422818792, 0.9142857142857143, ...","[0.0033464254621424277, 0.009536312664464652, ...","[0.0, 0.5302013422818792, 0.7039783001808317, ...","[0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, ...","[0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,B,"[C, C, C, C, C, C, C, C, C, C, C, C, C, A, C, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 4, 5, 5, 4, 5, 4, 5, 3, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",movie_recommendation,"[0.0, 0.8775510204081632, 0.8941798941798942, ...","[0.0033464254621424277, 0.015783816905508917, ...","[0.0, 0.8775510204081632, 0.8666879370396958, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [38]:
hard = pd.concat([hard,easy])
hard

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,SIM_COT_PW,SIM_AC_BIGRAM,SIM_AC_AGG,SIM_AC_PW,size_of_cot
0,0,A,"[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.9513888888888888, 0.8311688311688312, ...","[0.5, 0.9513888888888888, 0.8205128205128205, ...","[0.0, 0.9513888888888888, 0.8273491214667685, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
1,1,C,"[B, B, C, C, C, A, C, C, C, C, C, C, B, C, C, ...","[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1, 4, 3, 3, 4, 5, 3, 4, 3, 1, 3, 3, 4, 3, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 0, 0, 9, 0, 9, 2, 0, 2, 2, 0, 0, 2, 0, ...",disambiguation_qa,"[0.0, 0.5139664804469274, 0.6190476190476191, ...","[0.5, 0.5139664804469274, 0.5431472081218274, ...","[0.0, 0.5139664804469274, 0.5462102000563539, ...","[0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, ...","[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
2,2,C,"[ERROR, B, B, B, B, C, B, ERROR, B, B, B, B, B...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 4, 4, 4, 3, 4, 4, 0, 3, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 0, 0, 2, 0, 2, 9, 10, 0, 0, 0, 5, 0, 9, 0...",disambiguation_qa,"[0.0, 0.018292682926829285, 0.8596491228070176...","[0.5, 0.024390243902439046, 0.8546511627906976...","[0.0, 0.018292682926829285, 0.4394399460188934...","[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
3,3,C,"[C, B, C, C, B, C, C, C, C, B, B, B, C, C, C, ...","[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[1, 4, 1, 1, 4, 1, 1, 1, 1, 5, 4, 4, 1, 3, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.4204545454545454, 0.4204545454545454, ...","[0.5, 0.4204545454545454, 0.6079545454545454, ...","[0.0, 0.4204545454545454, 0.7102272727272727, ...","[0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, ...","[0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
4,4,C,"[C, C, C, C, C, C, C, C, C, C, C, C, B, A, C, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[4, 4, 5, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 5, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.8661971830985915, 0.6488095238095238, ...","[0.5, 0.8661971830985915, 0.6098901098901099, ...","[0.0, 0.8661971830985915, 0.6299603174603174, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,B,"[C, C, C, C, C, C, C, C, C, C, C, C, C, A, C, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 4, 5, 5, 4, 5, 4, 5, 3, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",movie_recommendation,"[0.0, 0.8775510204081632, 0.8941798941798942, ...","[0.0033464254621424277, 0.015783816905508917, ...","[0.0, 0.8775510204081632, 0.8666879370396958, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

# Early stopping and SC 

In [39]:
from collections import Counter

def calculate_correctness(df):
    # Define a helper function to determine the majority and compare it with the correct answer
    def check_majority(answers, correct):
        if not answers:
            return 0
        # Count the occurrences of each answer and find the most common one
        most_common = Counter(answers).most_common(1)[0][0]
        # Compare the most common answer with the correct answer
        return 1 if most_common == str(correct) else 0

    # Apply the helper function across the DataFrame rows
    df['SC_correctness'] = df.apply(lambda row: check_majority(row['CoT answers'], row['correct answer']), axis=1)
    return df

hard = calculate_correctness(hard)

In [40]:
hard.SC_correctness.value_counts()

SC_correctness
0    558
1    442
Name: count, dtype: int64

In [41]:
easy = calculate_correctness(easy)
easy.SC_correctness.value_counts()

SC_correctness
0    363
1    137
Name: count, dtype: int64

In [42]:
import pandas as pd

def evaluate_answers_with_window(df, window_size):
    # Helper function to evaluate answers with a sliding window
    def evaluate_window(answers, correct):
        # Track the number of steps (checks) taken
        steps = window_size-1
        
        # If the list is shorter than the window size, adjust the window size to the length of the list
        window_size_adjusted = min(window_size, len(answers))
        
        # Slide through the list with the adjusted window size
        for i in range(len(answers) - window_size_adjusted + 1):
            # Check the window content
            window = answers[i:i + window_size_adjusted]
            steps += 1
            
            # If all elements in the window are the same, evaluate correctness
            if window.count(window[0]) == window_size_adjusted:
                return 1 if window[0] == str(correct) else 0, steps
        
        # If no unanimous window is found, use the same value as SC_correctness and set steps to the length of the list
        majority = Counter(answers).most_common(1)[0][0]
        correctness = 1 if majority == str(correct) else 0
        return correctness, len(answers)

    # Apply the function to each row in the DataFrame
    result = df.apply(lambda row: evaluate_window(row['CoT answers'], row['correct answer']), axis=1)
    df['ES_correctness'] = result.apply(lambda x: x[0])
    df['ES_steps'] = result.apply(lambda x: x[1])

    return df

easy = evaluate_answers_with_window(easy,5)

In [43]:
easy.ES_correctness.value_counts()

ES_correctness
0    363
1    137
Name: count, dtype: int64

In [44]:
easy.ES_steps.mean()

16.204

In [45]:

hard = evaluate_answers_with_window(hard,5)
hard.ES_correctness.value_counts()

ES_correctness
0    557
1    443
Name: count, dtype: int64

In [46]:
hard.ES_steps.mean()

15.85

In [60]:
feature_li = [
 'LEN',
 'QUA_IM',
 'DIF_IV',
 # 'DIF_SUB',
 # 'SIM_COT_BIGRAM',
 'SIM_COT_AGG',
 # 'SIM_COT_PW',
 # 'SIM_AC_BIGRAM',
 'SIM_AC_AGG',
 # 'SIM_AC_PW',
    # 'size_of_cot'

 
  ]

# Visualize

In [61]:
import pandas as pd

def concatenate_columns(df,data_columns, outcome_column):
    # Initialize an empty dictionary to store the concatenated data
    concatenated_data = {}
    
    # Get the number of rows based on the length of the outcome column
    num_rows = len(df)
    
    # Iterate over each column
    for column in data_columns+[outcome_column]:
        # Initialize an empty list to store the concatenated values for the current column
        concatenated_values = []
        
        # Iterate over each row
        for i in range(num_rows):
            # Get the list of values for the current column and row
            
            values = df[column].iloc[i]
            
            # Concatenate the values into a single string
            concatenated_values += list(values)
            
        
        # Add the concatenated values to the dictionary with the column name as the key
        concatenated_data[column] = concatenated_values
        print(len(concatenated_values))
    # Add the outcome column to the concatenated data dictionary
    
    # Create a DataFrame from the concatenated data dictionary
    hard_final = pd.DataFrame(concatenated_data)
    
    return hard_final

hard_final = concatenate_columns(hard,feature_li,'Correctness')
easy_final = concatenate_columns(easy,feature_li,'Correctness')

40000
40000
40000
40000
40000
40000
20000
20000
20000
20000
20000
20000


In [62]:
hard_final['cot_answer'] = np.repeat(hard['correct answer'].values,40)

In [63]:
easy_final['cot_answer'] = np.repeat(easy['correct answer'].values,40)

In [64]:
from collections import Counter
print('--------------EASY--------------')
for col in easy_final.columns:
    if not col.startswith('SIM_COT_'):
        print(col, ':', Counter(easy_final[col]))
print('--------------HARD--------------')
for col in hard_final.columns:
    if not col.startswith('SIM_COT_'):
        print(col, ':', Counter(hard_final[col]))

--------------EASY--------------
LEN : Counter({4: 10910, 3: 4508, 5: 2228, 2: 853, 1: 558, 6: 431, 0: 284, 7: 161, 8: 43, 9: 12, 10: 8, 11: 3, 12: 1})
QUA_IM : Counter({0: 20000})
DIF_IV : Counter({0: 18318, 2: 1098, 26: 236, 5: 219, 9: 73, 10: 48, 1: 7, 3: 1})
SIM_AC_AGG : Counter({1: 13850, 0: 6150})
Correctness : Counter({0: 15115, 1: 4885})
cot_answer : Counter({'C': 5800, 'B': 5680, 'A': 5200, 'D': 3320})
--------------HARD--------------
LEN : Counter({4: 22566, 3: 8466, 5: 4341, 2: 1574, 1: 1232, 6: 806, 0: 593, 7: 270, 8: 115, 9: 21, 10: 11, 11: 4, 12: 1})
QUA_IM : Counter({0: 40000})
DIF_IV : Counter({0: 35127, 2: 3167, 5: 791, 10: 355, 9: 307, 26: 236, 1: 13, 11: 2, 3: 2})
SIM_AC_AGG : Counter({1: 27743, 0: 12257})
Correctness : Counter({0: 23952, 1: 16048})
cot_answer : Counter({'B': 11680, 'C': 11560, 'A': 10560, 'D': 5840, 'E': 320, 'NO': 40})


# CS based performance

In [65]:
def log_reg(features):
    coe = [-0.1, -5 , -1  , 3 , 2]
    lincomb = sum([coe[i]*features[i] for i in range(len(coe))]) -1.5
    return 1 / (1 + np.exp(-lincomb))
log_reg([5,0,0,1,1])

0.9525741268224334

In [66]:
df = hard
df_concate = hard_final
threshold = 0.36
warm_up_steps = 0

In [72]:
# df_concate['confidence_score']= cls_model.predict_proba(df_concate[feature_li])[:,1]
df_concate['confidence_score']=df_concate[feature_li].apply(log_reg,axis =1)
lists = [df_concate['confidence_score'].iloc[i:i+40].tolist() for i in range(0, len(df_concate), 40)]
df['confidence_score'] = lists

In [77]:
len(df_concate)

40000

In [73]:
def normalize_cs(cs_li, threshold):
    cs_arr = np.array(cs_li)
    # normalized_cs = [(cs-threshold)/(1-threshold) if cs > threshold else (cs-threshold)/(threshold) for cs in cs_arr]
    normalized_cs = cs_arr-threshold
    # normalized_cs = [0.5 if cs > threshold else -0.5 for cs in cs_arr]
    return np.array(normalized_cs)

In [74]:
def stop_con1(individual_cs):
    cumulative_difference = (individual_cs[warm_up_steps:]).cumsum()
    stop_idx = np.argmax(cumulative_difference > 0.5)
    return stop_idx
def stop_con2(individual_cs):
    buffer = []
    for idx,cs in enumerate(individual_cs):
        if cs>0:
            buffer.append(idx)
        if len(buffer) == 5:
            return buffer[-1]

In [75]:
CS_Answer = []
CS_correctness = []
CS_steps = []
for row_idx in range(len(df)): 
    test_row = df.iloc[row_idx]
    individual_cs = normalize_cs(test_row['confidence_score'],threshold)
    # individual_cs = test_row['confidence_score'][warm_up_steps:] - threshold
    stop_idx = stop_con2(individual_cs)
    
    if stop_idx:
        num_of_steps = warm_up_steps+stop_idx
    else: num_of_steps = 40
    answers = test_row['CoT answers'][:num_of_steps]
    scores = individual_cs[:num_of_steps]
    print(scores)
    weighted_votes = Counter()
    for answer, score in zip(answers, scores):
        if score >0:
            weighted_votes[answer] += score
    # Find the answer with the highest total score
    if (len(weighted_votes)) == 0:
        for answer, score in zip(answers, scores):
            weighted_votes[answer] += score
    print(weighted_votes)
    result = max(weighted_votes, key=weighted_votes.get)
    CS_Answer.append(result)
    CS_correctness.append(1 if result == str(test_row['correct answer']) else 0)
    CS_steps.append(num_of_steps)
df['CS_Answer'] = CS_Answer
df['CS_correctness'] = CS_correctness
df['CS_steps'] = CS_steps
df

[0.04131234 0.59046005 0.56834486 0.55049168]
Counter({'A': 1.7506089322603509})
[-0.25090318  0.05141944  0.09746347  0.16861192 -0.35984072  0.13775786
 -0.35902131]
Counter({'C': 0.26607539633488625, 'A': 0.13775786256265238, 'B': 0.051419444767485056})
[-0.3599546  -0.22138357  0.30014792  0.35284637  0.48004508 -0.25579364
 -0.35813946 -0.3599892   0.49070065]
Counter({'B': 1.6237400244546767})
[ 0.11502081 -0.01444514  0.5423721   0.5423721  -0.20108247  0.54244948]
Counter({'C': 1.7422144862377968})
[0.04131234 0.57694348 0.50172245 0.54860514]
Counter({'C': 1.6685834073851846})
[ 0.04131234 -0.35487634  0.50750424  0.20755316  0.52750618 -0.25895929]
Counter({'B': 1.0763227657153571, 'C': 0.20755315669776042})
[ 0.04131234  0.20605548  0.23402076  0.24904116 -0.19430052]
Counter({'A': 0.4830619243631423, 'C': 0.20605547958149262, 'B': 0.04131233988754801})
[0.04131234 0.47659674 0.5671994  0.52978741]
Counter({'B': 1.6148958879579451})
[0.04131234 0.18758284 0.53151233 0.527620

,id,correct answer,CoT answers,Correctness,LEN,QUA_IM,DIF_IV,DIF_SUB,SIM_COT_BIGRAM,SIM_COT_AGG,...,SIM_AC_AGG,SIM_AC_PW,size_of_cot,SC_correctness,ES_correctness,ES_steps,confidence_score,CS_Answer,CS_correctness,CS_steps
0,0,A,"[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.9513888888888888, 0.8311688311688312, ...","[0.5, 0.9513888888888888, 0.8205128205128205, ...",...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,5,"[0.401312339887548, 0.9504600476884206, 0.9283...",A,1,4
1,1,C,"[B, B, C, C, C, A, C, C, C, C, C, C, B, C, C, ...","[0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1, 4, 3, 3, 4, 5, 3, 4, 3, 1, 3, 3, 4, 3, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 0, 0, 9, 0, 9, 2, 0, 2, 2, 0, 0, 2, 0, ...",disambiguation_qa,"[0.0, 0.5139664804469274, 0.6190476190476191, ...","[0.5, 0.5139664804469274, 0.5431472081218274, ...",...,"[0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,11,"[0.10909682119561294, 0.41141944476748504, 0.4...",C,1,7
2,2,C,"[ERROR, B, B, B, B, C, B, ERROR, B, B, B, B, B...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 4, 4, 4, 3, 4, 4, 0, 3, 4, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 0, 0, 2, 0, 2, 9, 10, 0, 0, 0, 5, 0, 9, 0...",disambiguation_qa,"[0.0, 0.018292682926829285, 0.8596491228070176...","[0.5, 0.024390243902439046, 0.8546511627906976...",...,"[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",0,0,13,"[4.5397868702434395e-05, 0.13861643098453794, ...",B,0,9
3,3,C,"[C, B, C, C, B, C, C, C, C, B, B, B, C, C, C, ...","[1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[1, 4, 1, 1, 4, 1, 1, 1, 1, 5, 4, 4, 1, 3, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.4204545454545454, 0.4204545454545454, ...","[0.5, 0.4204545454545454, 0.6079545454545454, ...",...,"[0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,17,"[0.47502081252106, 0.3455548562725421, 0.90237...",C,1,6
4,4,C,"[C, C, C, C, C, C, C, C, C, C, C, C, B, A, C, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[4, 4, 5, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 5, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, ...",disambiguation_qa,"[0.0, 0.8661971830985915, 0.6488095238095238, ...","[0.5, 0.8661971830985915, 0.6098901098901099, ...",...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, ...","[3, 0, 1, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12...",1,1,5,"[0.401312339887548, 0.9369434833146464, 0.8617...",C,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,B,"[C, C, C, C, C, C, C, C, C, C, C, C, C, A, C, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 4, 4, 5, 5, 4, 5, 4, 5, 3, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",movie_recommendation,"[0.0, 0.8775510204081632, 0.8941798941798942, ...","[0.0033464254621424277, 0.015783816905508917, ...",...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, ...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,0,5,"[0.1202610177809003, 0.5367713537282465, 0.554...",C,0,5
496,496

In [71]:
print(df.ES_steps.mean(),df.CS_steps.mean())
print()
print(df.ES_correctness.value_counts(),df.CS_correctness.value_counts())
print()
print(df.ES_correctness.sum()/len(df),df.CS_correctness.sum()/len(df))

15.85 6.265

ES_correctness
0    557
1    443
Name: count, dtype: int64 CS_correctness
0    560
1    440
Name: count, dtype: int64

0.443 0.44


In [59]:

df_t07 = pd.read_json('../data/T0.7.jsonl', lines=True)

In [522]:
df_t07['generated_answer'].iloc[0][3]

"A: Janet's ducks lay 16 eggs per day. She eats 3 and uses 4 for muffins every day. So she has 16 - 3 - 4 = 9 eggs left to sell. She can sell the 9 eggs for 9 x 2 = $18 per day. The answer is $18."

In [1414]:
import re
ANS_RE = re.compile(r"#### (\-?[0-9\.\,]+)")
INVALID_ANS = "[invalid]"


def extract_answer(completion):
    match = ANS_RE.search(completion)
    if match:
        match_str = match.group(1).strip()
        match_str = match_str.replace(",", "")
        return match_str
    else:
        return INVALID_ANS
extract_answer(df_t07.iloc[0]['answer'])

'18'

In [1343]:
tem_1 = df_t07['generated_answer'].iloc[0][3]
number_list = re.findall(r"\d+\.?\d*", tem_1)
try:
    predict1 = number_list[-1].strip('.')
except:
    predict1 = -1000
predict1

'18'

In [937]:
number_list

['16', '3', '4', '16', '3', '4', '9', '9', '9', '2', '18', '18.']